In [1]:
#session_type = 'Technical_seminar'
session_type = 'BRK'
#session_type = 'Product-or-Strategy-Overview'
#session_type = 'Innovation_talk'

In [2]:
import json
import re

with open('credentials.json') as file:
    credentials = json.load(file)

In [3]:
import requests
from datetime import datetime, timedelta

def get_sessions(session_type, req_from = 0):
    '''
    Returns all sessions based on sessiontype.
    '''

    url = "https://events.rainfocus.com/api/search"

    payload =   {
                    'search.sessiontype': session_type,
                    'type': 'session',
                    'catalogDisplay': 'list',
                    'browserTimezone': 'Europe%2FParis',
                    'from': req_from
                }

    headers = {
        'authority': 'events.rainfocus.com',
        'accept': '*/*',
        'accept-language': 'en-US,en;q=0.9,fr-FR;q=0.8,fr;q=0.7',
        'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'origin': 'https://www.ciscolive.com',
        'referer': 'https://www.ciscolive.com/',
        'rfapiprofileid': credentials['rfapiprofileid'],
        'rfauthtoken': credentials['rfauthtoken'],
        'rfwidgetid': credentials['rfwidgetid'],
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    return response.json()

class Session():

    def __init__(self, session_json):
        self.type = session_json['type']
        try:
            self.start = datetime.strptime(session_json['times'][0]['utcStartTime'], '%Y/%m/%d %H:%M:%S') + timedelta(hours=1)
            self.end = datetime.strptime(session_json['times'][0]['utcEndTime'], '%Y/%m/%d %H:%M:%S') + timedelta(hours=1)
        except KeyError:
            self.start = 'Null'
            self.end = 'Null'
        self.day_name = session_json['times'][0]['dayName']
        self.name = re.sub(r"", "", session_json['title'])
        self.id = session_json['code']
        match int(self.id.split('-')[1][0]):
            case 1: self.level = 'Introductory'
            case 2: self.level = 'Intermediate'
            case 3: self.level = 'Advanced'
            case 4: self.level = 'General'
        self.capacity = int(session_json['times'][0]['capacity'])
        self.seats_remaining = int(session_json['times'][0]['seatsRemaining'])
        self.waitlist_remaining = int(session_json['times'][0]['waitlistRemaining'])
        self.room_id = session_json['times'][0]['roomId']
        self.room = session_json['times'][0]['room']
        
    def __str__(self):
        return self.name

In [4]:
sessions_json = get_sessions(session_type)

req_from = int(sessions_json['sectionList'][0]['from'])
num_items = int(sessions_json['sectionList'][0]['numItems'])
total = int(sessions_json['sectionList'][0]['total'])
size = int(sessions_json['sectionList'][0]['size'])

while req_from + num_items < total:
    req_from += size
    sessions_json_temp = get_sessions(session_type, req_from=req_from)

    for session in sessions_json_temp['items']:
        sessions_json['sectionList'][0]['items'].append(session)

KeyboardInterrupt: 

In [ ]:
sessions = []

for session in sessions_json['sectionList'][0]['items']:
    sessions.append(Session(session))

In [ ]:
import pandas as pd

In [ ]:
list_df = []

for session in sessions:

    percent_available = int((session.seats_remaining / session.capacity) * 100)

    if percent_available == 0: status = 'Full',
    elif percent_available > 20 : status = 'Overcapacity',
    else: status = 'Ok'

    to_append = {
        'Session Name': [session.id + ' - ' + session.name],
        'Day': session.day_name,
        'Timeslot': [session.start.strftime("%H:%M") + ' - ' + session.end.strftime("%H:%M")],
        'Total Capacity': session.capacity,
        'Empty Seats': session.seats_remaining, 
        '% Available': [percent_available],
        'Room': session.room,
        'Status': status,
    }
    
    list_df.append(pd.DataFrame(data=to_append))

df = pd.concat(list_df, names=['Session Name', 'Day', 'Timeslot', 'Total Capacity', 'Empty Seats', '% Available', 'Room', 'Status'])

ValueError: No objects to concatenate

In [ ]:
df.to_excel('open_' + session_type + '.xlsx', index=False)

In [ ]:
full_sessions = 0

for session in sessions:
    if session.seats_remaining < 1:
        full_sessions += 1

percentage_sessions_full = int((full_sessions / sessions.__len__()) * 100)

In [ ]:
print(percentage_sessions_full, 'percent of', session_type, 'are full!')

25 percent of BRK are full!


In [ ]:
sessions_json

{'responseCode': '0',
 'responseMessage': 'Success',
 'totalSearchItems': 0,
 'sections': True,
 'sectionList': [{'sectionId': '1',
   'sectionTitle': '',
   'total': 0,
   'numItems': 0,
   'from': 0,
   'size': 50,
   'items': [],
   'dateHistograms': {}}],
 'attributes': [{'id': 'track',
   'name': 'Track',
   'rf_attribute_id': '1415827519915001KuSb',
   'displayorder': 1.0,
   'dataForm': '16559055579890125AZP',
   'viewAccess': [],
   'scheduleAccess': [],
   'published': 1.0,
   'viewAccessPublic': True,
   'values': [{'rf_attributevalue_id': '1611355117405001yk3B',
     'rf_attribute_id': '1415827519915001KuSb',
     'attributevalue_code': 'track_app',
     'dataForm': '16559055579890125AZP',
     'published': 1.0,
     'viewAccessPublic': True,
     'viewAccess': [],
     'scheduleAccess': ['1668175714784001fU5L',
      '1668795876108001qyPs',
      '1668177921903001Hioi',
      '1668187083366001LXMP'],
     'attribute': 'Track',
     'displayorder': 2.0,
     'id': 'track_app

In [ ]:
sessions_json['sectionList'][0]

{'sectionId': '1',
 'sectionTitle': '',
 'total': 0,
 'numItems': 0,
 'from': 0,
 'size': 50,
 'items': [],
 'dateHistograms': {}}